In [1]:
!pip install pytorch_pretrained_bert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 4.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manyl

In [2]:
import numpy.linalg._umath_linalg
print(numpy.linalg._umath_linalg.__file__)

/usr/local/lib/python3.10/dist-packages/numpy/linalg/_umath_linalg.cpython-310-x86_64-linux-gnu.so


In [3]:
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']

import random
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import regex as re
#from pymagnitude import Magnitude
from collections.abc import MutableMapping
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification, BertAdam
from sklearn.metrics import classification_report
from tqdm import tqdm, tqdm_notebook

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader, Subset
from torch.utils.data.dataset import random_split
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

from IPython.core.display import display, HTML
from google_drive_downloader import GoogleDriveDownloader as gdd

tqdm.pandas()

# If the machine you run this on has a GPU available with CUDA installed,
# use it. Using a GPU for learning often leads to huge speedups in training.
# See https://developer.nvidia.com/cuda-downloads for installing CUDA
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


### 1) Loading the SQUAD 2.0

In [5]:
from datasets import load_dataset

# Load SQuAD 2.0 dataset
squad = load_dataset("squad_v2")

# Access training and validation splits
train_dataset = squad["train"]
validation_dataset = squad["validation"]
print(len(train_dataset))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

130319


In [6]:
train_set = []
val_set = []
l = list(range(1,len(train_dataset)))
k = list(range(1,len(validation_dataset)))
random.shuffle(l)
random.shuffle(k)
# Access training and validation splits
for i in range(500):
    if (train_dataset[l[i]]['answers']['text']):
        train_set.append(train_dataset[l[i]])

for i in range(500):
    if (validation_dataset[k[i]]['answers']['text']):
        val_set.append(validation_dataset[k[i]])

print("Total length of QnA pairs in Train dataset used: "+str(len(train_set)))
print("Total length of QnA pairs in Val dataset used: "+str(len(val_set)))

Total length of QnA pairs in Train dataset used: 340
Total length of QnA pairs in Val dataset used: 258


In [7]:
#Raw Samples
for i in range(10):
    print(train_set[i])

{'id': '5727c4424b864d1900163cce', 'title': 'Detroit', 'context': 'Long a major population center and site of worldwide automobile manufacturing, Detroit has suffered a long economic decline produced by numerous factors. Like many industrial American cities, Detroit reached its population peak in the 1950 census. The peak population was 1.8 million people. Following suburbanization, industrial restructuring, and loss of jobs (as described above), by the 2010 census, the city had less than 40 percent of that number, with just over 700,000 residents. The city has declined in population in each census since 1950.', 'question': 'By how much in percent has the population of Detroit declined since 1950?', 'answers': {'text': ['40'], 'answer_start': [427]}}
{'id': '570fd63b80d9841400ab36ae', 'title': 'Dell', 'context': 'Dell has been attempting to offset its declining PC business, which still accounted for half of its revenue and generates steady cash flow, by expanding into the enterprise ma

### 2) Displaying a few Question and Answer pairs in the dataset¶

In [8]:
for i in range(10):
    sample = train_set[i]
    question = sample['question']
    answer = sample['answers']
    print(f"Question: {question}\nAnswer: {answer}\n")

Question: By how much in percent has the population of Detroit declined since 1950?
Answer: {'text': ['40'], 'answer_start': [427]}

Question: Who was Dell's main business rival?
Answer: {'text': ['Hewlett Packard'], 'answer_start': [329]}

Question: Who was reigning when Henry Yevele finished his work on the abbey?
Answer: {'text': ['Richard II'], 'answer_start': [524]}

Question: What did Trump Entertainment Group say they were considering doing with the Trump Taj Mahal?
Answer: {'text': ['selling'], 'answer_start': [156]}

Question: What is the first degree of Masonry?
Answer: {'text': ['Entered Apprentice'], 'answer_start': [154]}

Question: The compression used in all HDTV storage and transmission systems will do what if resolution is too high?
Answer: {'text': ['distort the received picture'], 'answer_start': [218]}

Question: How much heat is required for the natural production of asphalt?
Answer: {'text': ['above 50 °C'], 'answer_start': [416]}

Question: What did Darwin's cont

### 3. Preprocess/Clean dataset as per BERT format.

In [9]:
train_answers = []
train_texts = []
train_queries = []

for i in range(len(train_set)):
    train_answers.append(train_set[i]['answers'])
    train_texts.append(train_set[i]['context'])
    train_queries.append(train_set[i]['question'])

In [10]:
val_answers = []
val_texts = []
val_queries = []

for i in range(len(val_set)):
    val_answers.append(val_set[i]['answers'])
    val_texts.append(val_set[i]['context'])
    val_queries.append(val_set[i]['question'])

In [11]:
for answer, text in zip(train_answers, train_texts):
    answer['text'] = answer['text'][0]
    answer['answer_start'] = answer['answer_start'][0]
    real_answer = answer['text']
    start_idx = answer['answer_start']
    # Get the real end index
    end_idx = start_idx + len(real_answer)

    # Deal with the problem of 1 or 2 more characters
    if text[start_idx:end_idx] == real_answer:
        answer['answer_end'] = end_idx
    # When the real answer is more by one character
    elif text[start_idx-1:end_idx-1] == real_answer:
        answer['answer_start'] = start_idx - 1
        answer['answer_end'] = end_idx - 1
    # When the real answer is more by two characters
    elif text[start_idx-2:end_idx-2] == real_answer:
        answer['answer_start'] = start_idx - 2
        answer['answer_end'] = end_idx - 2



In [12]:
for answer, text in zip(val_answers, val_texts):
    answer['text'] = answer['text'][0]
    answer['answer_start'] = answer['answer_start'][0]
    real_answer = answer['text']
    start_idx = answer['answer_start']
    # Get the real end index
    end_idx = start_idx + len(real_answer)

    # Deal with the problem of 1 or 2 more characters
    if text[start_idx:end_idx] == real_answer:
        answer['answer_end'] = end_idx
    # When the real answer is more by one character
    elif text[start_idx-1:end_idx-1] == real_answer:
        answer['answer_start'] = start_idx - 1
        answer['answer_end'] = end_idx - 1
    # When the real answer is more by two characters
    elif text[start_idx-2:end_idx-2] == real_answer:
        answer['answer_start'] = start_idx - 2
        answer['answer_end'] = end_idx - 2

In [13]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [14]:
train_encodings = tokenizer(train_texts, train_queries, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, val_queries, truncation=True, padding=True)

In [15]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []

    count = 0

    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length

        # if end position is None, the 'char_to_token' function points to the space after the correct token, so add - 1
        if end_positions[-1] is None:
            end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end'] - 1)

            # if end position is still None the answer passage has been truncated
            if end_positions[-1] is None:
                count += 1
                end_positions[-1] = tokenizer.model_max_length
    print(count)

    # Update the data in dictionary
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)

0
0


In [16]:
class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

In [17]:
train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)

In [18]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

In [19]:
device = torch.device('cuda:0' if torch.cuda.is_available()
                      else 'cpu')
print(device)

cuda:0


In [20]:
!pip install transformers
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased').to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 4. Train the BERT QnA model. Evaluate the model.

In [21]:
from transformers import AdamW
optim = AdamW(model.parameters(), lr=5e-5)

epochs = 5

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [22]:
import time
whole_train_eval_time = time.time()

train_losses = []
val_losses = []

print_every = 1000

for epoch in range(epochs):
    epoch_time = time.time()

    # Set model in train mode
    model.train()
    loss_of_epoch = 0

    print("TRAIN")

    for batch_idx,batch in enumerate(train_loader):
        optim.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs[0]
        # do a backwards pass
        loss.backward()
        # update the weights
        optim.step()
        # Find the total loss
        loss_of_epoch += loss.item()

        if (batch_idx+1) % print_every == 0:
            print("Batch {:} / {:}".format(batch_idx+1,len(train_loader)),"\nLoss:", round(loss.item(),1),"\n")

    loss_of_epoch /= len(train_loader)
    train_losses.append(loss_of_epoch)

   # Set model in evaluation mode
    model.eval()

    print("EVALUATE")

    loss_of_epoch = 0

    for batch_idx,batch in enumerate(val_loader):

        with torch.no_grad():

            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            start_positions = batch['start_positions'].to(device)
            end_positions = batch['end_positions'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
            loss = outputs[0]
            # Find the total loss
            loss_of_epoch += loss.item()

        if (batch_idx+1) % print_every == 0:
            print("Batch {:} / {:}".format(batch_idx+1,len(val_loader)),"\nLoss:", round(loss.item(),1),"\n")

    loss_of_epoch /= len(val_loader)
    val_losses.append(loss_of_epoch)

    # Print each epoch's time and train/val loss

    print("\n-------Epoch ", epoch+1,
          "-------"
          "\nTraining Loss:", train_losses[-1],
          "\nValidation Loss:", val_losses[-1],
          "\nTime: ",(time.time() - epoch_time),
          "\n-----------------------",
          "\n\n")

print("Total training and evaluation time: ", (time.time() - whole_train_eval_time))

TRAIN
EVALUATE

-------Epoch  1 -------
Training Loss: 4.640202078708382 
Validation Loss: 4.2269791039553555 
Time:  21.284015655517578 
----------------------- 


TRAIN
EVALUATE

-------Epoch  2 -------
Training Loss: 3.4613445304160892 
Validation Loss: 3.9760568141937256 
Time:  20.726964235305786 
----------------------- 


TRAIN
EVALUATE

-------Epoch  3 -------
Training Loss: 2.198679732721905 
Validation Loss: 4.454829216003418 
Time:  21.14414143562317 
----------------------- 


TRAIN
EVALUATE

-------Epoch  4 -------
Training Loss: 0.9696812962376794 
Validation Loss: 5.1890730135368575 
Time:  21.689436674118042 
----------------------- 


TRAIN
EVALUATE

-------Epoch  5 -------
Training Loss: 0.5433000073183415 
Validation Loss: 5.389322230310151 
Time:  21.961572647094727 
----------------------- 


Total training and evaluation time:  106.819415807724


### 5. Perform an Inference and show the predicted vs ground truth answers.

In [45]:
def predict(context,query):

    inputs = tokenizer.encode_plus(query, context, return_tensors='pt').to(device)
    outputs = model(**inputs)
    answer_start = torch.argmax(outputs[0])  # get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(outputs[1]) + 1
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
    return answer

def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re
    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()

    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)

    common_tokens = set(pred_tokens) & set(truth_tokens)

    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0

    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    return 2 * (prec * rec) / (prec + rec)

In [69]:
from transformers import pipeline

# Initialize the question-answering model
qa_pipeline = pipeline('question-answering', model='bert-large-uncased-whole-word-masking-finetuned-squad')

def predict_answer(context, question):
    try:
        result = qa_pipeline(question=question, context=context)
        return result['answer']
    except Exception as e:
        print(f"Error during prediction: {e}")
        return ""


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [70]:
def give_an_answer(context, query, answer):
    prediction = predict_answer(context, query)
    print(f"Debug - Prediction: {prediction}")  # Debug print to check what is being predicted
    em_score = compute_exact_match(prediction, answer)
    f1_score = compute_f1(prediction, answer)
    print(f"Question: {query}")
    print(f"Prediction: {prediction}")
    print(f"True Answer: {answer}")
    print(f"Exact Match: {em_score}, F1: {f1_score}")
    print("\n")


In [71]:
context = "The Great Wall of China, one of the greatest wonders of the world, was built to protect and consolidate territories of Chinese states and empires against various nomadic groups of the steppe regions. It is a series of fortifications made of stone, brick, tamped earth, wood, and other materials, generally built along an east-to-west line across the historical northern borders of China."

# Define questions and answers
queries = [
    "What was the purpose of the Great Wall of China?",
    "What materials were used to build the Great Wall of China?",
    "Where is the Great Wall of China generally located?"
]

answers = [
    "To protect and consolidate territories of Chinese states and empires against various nomadic groups.",
    "Stone, brick, tamped earth, wood, and other materials.",
    "Along an east-to-west line across the historical northern borders of China."
]

for q, a in zip(queries, answers):
    give_an_answer(context, q, a)

Debug - Prediction: to protect and consolidate territories
Question: What was the purpose of the Great Wall of China?
Prediction: to protect and consolidate territories
True Answer: To protect and consolidate territories of Chinese states and empires against various nomadic groups.
Exact Match: 0, F1: 0.5263157894736842


Debug - Prediction: stone, brick, tamped earth, wood, and other materials
Question: What materials were used to build the Great Wall of China?
Prediction: stone, brick, tamped earth, wood, and other materials
True Answer: Stone, brick, tamped earth, wood, and other materials.
Exact Match: 1, F1: 1.0


Debug - Prediction: northern borders of China
Question: Where is the Great Wall of China generally located?
Prediction: northern borders of China
True Answer: Along an east-to-west line across the historical northern borders of China.
Exact Match: 0, F1: 0.6153846153846153




In [73]:
context = "Artificial intelligence (AI) has rapidly advanced over the past decade, influencing various sectors such as healthcare, finance, and transportation. AI technologies use machine learning algorithms to process vast amounts of data, learn patterns, and make decisions with minimal human intervention. This technological revolution has led to significant improvements in efficiency and has opened up new possibilities for innovation."


queries = [
        "What sectors has AI significantly influenced?",
        "How do AI technologies function?",
        "What are the benefits of AI in modern industries?"
    ]
answers = [
        "Healthcare, finance, and transportation",
        "They use machine learning algorithms to process data, learn patterns, and make decisions with minimal human intervention",
        "Improvements in efficiency and new possibilities for innovation"
    ]

for q,a in zip(queries,answers):
    give_an_answer(context,q,a)


Debug - Prediction: healthcare, finance, and transportation
Question: What sectors has AI significantly influenced?
Prediction: healthcare, finance, and transportation
True Answer: Healthcare, finance, and transportation
Exact Match: 1, F1: 1.0


Debug - Prediction: use machine learning algorithms
Question: How do AI technologies function?
Prediction: use machine learning algorithms
True Answer: They use machine learning algorithms to process data, learn patterns, and make decisions with minimal human intervention
Exact Match: 0, F1: 0.38095238095238093


Debug - Prediction: significant improvements in efficiency and has opened up new possibilities for innovation
Question: What are the benefits of AI in modern industries?
Prediction: significant improvements in efficiency and has opened up new possibilities for innovation
True Answer: Improvements in efficiency and new possibilities for innovation
Exact Match: 0, F1: 0.8




In [77]:
context = "Hello, I'm Jordan and I'm 29 years old. I graduated from the University of Cambridge, and I currently reside in the city of Bristol, UK. Previously, I lived in London for several years during my studies."

# Questions about Jordan
queries = ["How old is Jordan?",
           "Where does Jordan live now?",
           "Where did Jordan live before moving to his current location?"]

# Corresponding answers to the queries
answers = ["29",
           "Bristol",
           "London"]

# Loop through each question and answer, applying the function to give an answer
for q, a in zip(queries, answers):
    give_an_answer(context, q, a)

Debug - Prediction: 29
Question: How old is Jordan?
Prediction: 29
True Answer: 29
Exact Match: 1, F1: 1.0


Debug - Prediction: Bristol
Question: Where does Jordan live now?
Prediction: Bristol
True Answer: Bristol
Exact Match: 1, F1: 1.0


Debug - Prediction: London
Question: Where did Jordan live before moving to his current location?
Prediction: London
True Answer: London
Exact Match: 1, F1: 1.0




1. **F1 Score**: This metric quantifies the accuracy of the predictions, taking into account both precision (what proportion of predicted tokens were correct) and recall (what proportion of actual tokens were predicted correctly). The F1 score is particularly useful for assessing answers where partial matches occur, as it reflects the overall overlap between the predicted and true answers.

2. **Exact Match**: This is a stricter metric that checks if the predicted answer matches the true answer exactly, without any deviations. An Exact Match score of 1 indicates a perfect match, whereas 0 indicates that there was some discrepancy between the predicted and actual answers.

**Performance Analysis**:
- The model achieved perfect Exact Match and F1 scores of 1.0 for several questions, indicating precise predictions, particularly when the answers were straightforward and factual, such as ages, locations, or clear definitions.
- For questions requiring interpretation or when the answer involved more complex phrasing, the F1 scores varied, such as:
  - 0.526 for the purpose of the Great Wall of China, indicating moderate overlap.
  - 0.615 for the location description of the Great Wall, reflecting a reasonable, but not complete, match.
  - Lower F1 scores (0.380) in cases like explaining the functioning of AI technologies, suggesting that while the model could identify relevant concepts (like "machine learning algorithms"), it missed the fuller context needed for a complete answer.

Overall, the F1 and Exact Match metrics provided a nuanced view of the model's ability to handle different types of questions, from direct facts to interpretative answers, highlighting areas where the model performs well and where improvements might be needed for better understanding or more detailed answer generation.